In [102]:
import pandas as pd
import torch
# data duration: 210101-210630 / 181 days

In [121]:
# import needed data
temperature = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_3시간기온_202101_202112.csv', encoding='cp949')
temperature_ = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_1시간기온_20210701_20211231.csv', encoding='cp949')


# refine temperature data
# temperature(_): forecast point: 23pm, forecast horizon: 7,10,13,16,19h(6,9,12,15,18), I don't care 3, 21, 24 where pv generation doesn't occur
temperature = temperature.loc[(temperature['hour'] == 2300.0) & ((temperature['forecast'] == 7.0) | (temperature['forecast'] == 10.0)
                              | (temperature['forecast'] == 13.0) | (temperature['forecast'] == 16.0) | (temperature['forecast'] == 19.0))]
temperature.columns = ['day', 'forecast point', 'forecast horizon', 'temperature']
temperature = temperature.reset_index(drop = True).set_index('day').drop(['forecast point', 'forecast horizon'], axis = 1)
temp_tnsr = torch.FloatTensor(temperature.values) # 905 rows
temp_tnsr = temp_tnsr.reshape(181,5)  # 905/5=181
temp_new_df = pd.DataFrame(temp_tnsr, columns = ['6','9','12','15','18'])  # set column names as forecasted time point


# refine temperature_ data
# (1) temperature_ file has many strange entities and (2) the same data is repeated three times
# (1)
del_index = []
for i, l in enumerate(temperature_[' format: day']):
    # if ' Start' in l:
    # if 'Start' in l or 'format' in l:   # There are cells filled with ' Start: ...' and 'format: ...'
    #     print(i, l)
    #     del_index.append(i)
    try:
        float(l)
    except ValueError:
        del_index.append(i)
# (1)//  
temperature_.columns = ['day', 'forecast point', 'forecast horizon', 'temperature']
temperature_ = temperature_.drop(del_index).reset_index(drop = True).astype('float')
temperature_ = temperature_.loc[(temperature_['forecast point'] == 2300.0) & ((temperature_['forecast horizon'] == 7.0) | (temperature_['forecast horizon'] == 10.0)
                              | (temperature_['forecast horizon'] == 13.0) | (temperature_['forecast horizon'] == 16.0) | (temperature_['forecast horizon'] == 19.0))].reset_index(drop = True)
# (2)
temperature_ = temperature_[['temperature']].iloc[0:920]  # data is duplicated three times, so only 1/3 is picked   # 7~12 month have 184 days, so the number of rows is 184*5=920
# (2)//
temp_tnsr = torch.FloatTensor(temperature_.values) # 920 rows
temp_tnsr = temp_tnsr.reshape(184,5)  # 920/5=184
temp_new_df_ = pd.DataFrame(temp_tnsr, columns = ['6','9','12','15','18'])  # set column names as forecasted time point


# combine df of temperature and temperature_
temperature = temp_new_df.append(temp_new_df_).reset_index(drop=True)
temperature.columns = ['TM_6','TM_9','TM_12','TM_15','TM_18']

C:\Users\user\AppData\Local\Temp\ipykernel_6916\3013681990.py:3: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  temperature_ = pd.read_csv('./weather_forecast_data/첨단1동_1시간기온_20210701_20211231.csv', encoding='cp949')
C:\Users\user\AppData\Local\Temp\ipykernel_6916\3013681990.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temperature = temp_new_df.append(temp_new_df_).reset_index(drop=True)


In [122]:
# import needed data
precipitation = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_6시간강수량_202101_202112.csv', names=['day', 'forecast point', 'forecast horizon', 'precipitation'], encoding='cp949')
precipitation_ = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_1시간강수량_20210701_20211231.csv', names=['day', 'forecast point', 'forecast horizon', 'precipitation'], encoding='cp949')


# refine precipitation data
# precipitation: forecast point: 23pm, forecast horizon: 10,16,22(9,15,21) precipitation data starts after 4 hours and has 6h interval, and precipitation_ data starts after 6 hours, so after 10h
#                is the first time of the intersection
precipitation = precipitation.loc[(precipitation['forecast point'] == 2300.0) & ((precipitation['forecast horizon'] == 10.0) | (precipitation['forecast horizon'] == 16.0)
                              | (precipitation['forecast horizon'] == 22.0))]
precipitation_ = precipitation_.loc[(precipitation_['forecast point'] == 2300.0) & ((precipitation_['forecast horizon'] == 10.0) | (precipitation_['forecast horizon'] == 16.0)
                              | (precipitation_['forecast horizon'] == 22.0))]
precipitation_ = precipitation_.iloc[0:552]

precipitation = precipitation.reset_index(drop = True)[['precipitation']]
precip_tnsr = torch.FloatTensor(precipitation.values) # 543 rows
precip_tnsr = precip_tnsr.reshape(181,3)  # 543/3=181
precip_new_df = pd.DataFrame(precip_tnsr, columns = ['9','15','21'])  # set column names as forecasted time point

precipitation_ = precipitation_.reset_index(drop = True)[['precipitation']]
precip_tnsr_ = torch.FloatTensor(precipitation_.values) # 552 rows
precip_tnsr_ = precip_tnsr_.reshape(184,3)  # 552/3=184
precip_new_df_ = pd.DataFrame(precip_tnsr_, columns = ['9','15','21'])  # set column names as forecasted time point


# combine df of temperature and temperature_
precipitation = precip_new_df.append(precip_new_df_).reset_index(drop=True)
precipitation.columns = ['PR_9','PR_15','PR_21']

C:\Users\user\AppData\Local\Temp\ipykernel_6916\355438290.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precipitation = precip_new_df.append(precip_new_df_).reset_index(drop=True)


In [124]:
# import needed data
prob_precip = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_강수확률_202101_202112.csv', names=['day', 'forecast point', 'forecast horizon', 'probability'], encoding='cp949')
prob_precip_ = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_강수확률_20210701_20211231.csv', names=['day', 'forecast point', 'forecast horizon', 'probability'], encoding='cp949')



# refine precipitation probability data
# precipitation probability: forecast point: 23pm, forecast horizon: 7,10,13,16,19(6,9,12,15,18)
prob_precip = prob_precip.loc[(prob_precip['forecast point'] == 2300.0) & ((prob_precip['forecast horizon'] == 7.0) | (prob_precip['forecast horizon'] == 10.0)
                              | (prob_precip['forecast horizon'] == 13.0) | (prob_precip['forecast horizon'] == 16.0) | (prob_precip['forecast horizon'] == 19.0))]
prob_precip_ = prob_precip_.loc[(prob_precip_['forecast point'] == 2300.0) & ((prob_precip_['forecast horizon'] == 7.0) | (prob_precip_['forecast horizon'] == 10.0)
                              | (prob_precip_['forecast horizon'] == 13.0) | (prob_precip_['forecast horizon'] == 16.0) | (prob_precip_['forecast horizon'] == 19.0))]


prob_precip = prob_precip.iloc[0:905]
prob_precip_ = prob_precip_.iloc[0:920]

prob_precip = prob_precip.reset_index(drop = True)[['probability']]
prob_tnsr = torch.FloatTensor(prob_precip.values) # 905 rows
prob_tnsr  = prob_tnsr .reshape(181,5)  # 905/5=181
prob_new_df = pd.DataFrame(prob_tnsr, columns = ['6','9','12','15','18'])  # set column names as forecasted time point

prob_precip_ = prob_precip_.reset_index(drop = True)[['probability']]
prob_tnsr_ = torch.FloatTensor(prob_precip_.values) # 920 rows
prob_tnsr_ = prob_tnsr_.reshape(184,5)  # 920/5=184
prob_new_df_ = pd.DataFrame(prob_tnsr_, columns = ['6','9','12','15','18'])  # set column names as forecasted time point


# combine df of temperature and temperature_
prob_precip = prob_new_df.append(prob_new_df_).reset_index(drop=True)
prob_precip.columns = ['PP_6','PP_9','PP_12','PP_15','PP_18']

C:\Users\user\AppData\Local\Temp\ipykernel_6916\2135893994.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prob_precip = prob_new_df.append(prob_new_df_).reset_index(drop=True)


In [127]:
humidity = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_습도_202101_202112.csv', names=['day', 'forecast point', 'forecast horizon', 'humidity'], encoding='cp949')
humidity_ = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_습도_20210701_20211231.csv', names=['day', 'forecast point', 'forecast horizon', 'humidity'], encoding='cp949')


# 둘 다 두 번 반복됨


# refine precipitation probability data
# precipitation probability: forecast point: 23pm, forecast horizon: 7,10,13,16,19(6,9,12,15,18)
humidity = humidity.loc[(humidity['forecast point'] == 2300.0) & ((humidity['forecast horizon'] == 7.0) | (humidity['forecast horizon'] == 10.0)
                              | (humidity['forecast horizon'] == 13.0) | (humidity['forecast horizon'] == 16.0) | (humidity['forecast horizon'] == 19.0))]
humidity_ = humidity_.loc[(humidity_['forecast point'] == 2300.0) & ((humidity_['forecast horizon'] == 7.0) | (humidity_['forecast horizon'] == 10.0)
                              | (humidity_['forecast horizon'] == 13.0) | (humidity_['forecast horizon'] == 16.0) | (humidity_['forecast horizon'] == 19.0))]


humidity = humidity.iloc[0:905]
humidity_ = humidity_.iloc[0:920]

humidity = humidity.reset_index(drop = True)[['humidity']]
hum_tnsr = torch.FloatTensor(humidity.values) # 905 rows
hum_tnsr  = hum_tnsr.reshape(181,5)  # 905/5=181
hum_new_df = pd.DataFrame(hum_tnsr, columns = ['6','9','12','15','18'])  # set column names as forecasted time point

humidity_ = humidity_.reset_index(drop = True)[['humidity']]
hum_tnsr_ = torch.FloatTensor(humidity_.values) # 920 rows
hum_tnsr_ = hum_tnsr_.reshape(184,5)  # 920/5=184
hum_new_df_ = pd.DataFrame(hum_tnsr_, columns = ['6','9','12','15','18'])  # set column names as forecasted time point


# combine df of temperature and temperature_
humidity = hum_new_df.append(hum_new_df_).reset_index(drop=True)
humidity.columns = ['HM_6','HM_9','HM_12','HM_15','HM_18']

C:\Users\user\AppData\Local\Temp\ipykernel_6916\975514263.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  humidity_ = pd.read_csv('./weather_forecast_data/첨단1동_습도_20210701_20211231.csv', names=['day', 'forecast point', 'forecast horizon', 'humidity'], encoding='cp949')
C:\Users\user\AppData\Local\Temp\ipykernel_6916\975514263.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  humidity = hum_new_df.append(hum_new_df_).reset_index(drop=True)


In [128]:
wind_speed = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_풍속_202101_202112.csv', names=['day', 'forecast point', 'forecast horizon', 'wind speed'], encoding='cp949')
wind_speed_ = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_풍속_20210701_20211231.csv', names=['day', 'forecast point', 'forecast horizon', 'wind speed'], encoding='cp949')


# refine precipitation probability data
# precipitation probability: forecast point: 23pm, forecast horizon: 7,10,13,16,19(6,9,12,15,18)
wind_speed = wind_speed.loc[(wind_speed['forecast point'] == 2300.0) & ((wind_speed['forecast horizon'] == 7.0) | (wind_speed['forecast horizon'] == 10.0)
                              | (wind_speed['forecast horizon'] == 13.0) | (wind_speed['forecast horizon'] == 16.0) | (wind_speed['forecast horizon'] == 19.0))]
wind_speed_ = wind_speed_.loc[(wind_speed_['forecast point'] == 2300.0) & ((wind_speed_['forecast horizon'] == 7.0) | (wind_speed_['forecast horizon'] == 10.0)
                              | (wind_speed_['forecast horizon'] == 13.0) | (wind_speed_['forecast horizon'] == 16.0) | (wind_speed_['forecast horizon'] == 19.0))]


wind_speed = wind_speed.iloc[0:905]

wind_speed = wind_speed.reset_index(drop = True)[['wind speed']]
ws_tnsr = torch.FloatTensor(wind_speed.values) # 905 rows
ws_tnsr  = ws_tnsr.reshape(181,5)  # 905/5=181
ws_new_df = pd.DataFrame(ws_tnsr, columns = ['6','9','12','15','18'])  # set column names as forecasted time point

wind_speed_ = wind_speed_.reset_index(drop = True)[['wind speed']]
ws_tnsr_ = torch.FloatTensor(wind_speed_.values) # 920 rows
ws_tnsr_ = ws_tnsr_.reshape(184,5)  # 920/5=184
ws_new_df_ = pd.DataFrame(ws_tnsr_, columns = ['6','9','12','15','18'])  # set column names as forecasted time point


# combine df of temperature and temperature_
wind_speed = ws_new_df.append(ws_new_df_).reset_index(drop=True)
wind_speed.columns = ['WS_6','WS_9','WS_12','WS_15','WS_18']

C:\Users\user\AppData\Local\Temp\ipykernel_6916\2963778742.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wind_speed = ws_new_df.append(ws_new_df_).reset_index(drop=True)


In [129]:
sky = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_하늘상태_202101_202112.csv', names=['day', 'forecast point', 'forecast horizon', 'sky'], encoding='cp949')
sky_ = pd.read_csv('./preprocessed_data/weather_forecast_data/첨단1동_하늘상태_20210701_20211231.csv', names=['day', 'forecast point', 'forecast horizon', 'sky'], encoding='cp949')


# refine precipitation probability data
# precipitation probability: forecast point: 23pm, forecast horizon: 7,10,13,16,19(6,9,12,15,18)
sky = sky.loc[(sky['forecast point'] == 2300.0) & ((sky['forecast horizon'] == 7.0) | (sky['forecast horizon'] == 10.0)
                              | (sky['forecast horizon'] == 13.0) | (sky['forecast horizon'] == 16.0) | (sky['forecast horizon'] == 19.0))]
sky_ = sky_.loc[(sky_['forecast point'] == 2300.0) & ((sky_['forecast horizon'] == 7.0) | (sky_['forecast horizon'] == 10.0)
                              | (sky_['forecast horizon'] == 13.0) | (sky_['forecast horizon'] == 16.0) | (sky_['forecast horizon'] == 19.0))]


sky_ = sky_.iloc[0:920]

sky = sky.reset_index(drop = True)[['sky']]
sky_tnsr = torch.FloatTensor(sky.values) # 905 rows
sky_tnsr  = sky_tnsr.reshape(181,5)  # 905/5=181
sky_new_df = pd.DataFrame(sky_tnsr, columns = ['6','9','12','15','18'])  # set column names as forecasted time point

sky_ = sky_.reset_index(drop = True)[['sky']]
sky_tnsr_ = torch.FloatTensor(sky_.values) # 920 rows
sky_tnsr_ = sky_tnsr_.reshape(184,5)  # 920/5=184
sky_new_df_ = pd.DataFrame(sky_tnsr_, columns = ['6','9','12','15','18'])  # set column names as forecasted time point


# combine df of temperature and temperature_
sky = sky_new_df.append(sky_new_df_).reset_index(drop=True)
sky.columns = ['SK_6','SK_9','SK_12','SK_5','SK_18']

C:\Users\user\AppData\Local\Temp\ipykernel_6916\883522756.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sky = sky_new_df.append(sky_new_df_).reset_index(drop=True)


In [228]:
weather_forecast_original_df = pd.concat([temperature, precipitation, prob_precip, humidity, wind_speed, sky], axis=1)


In [229]:
weather_forecast_original_df.columns

Index(['TM_6', 'TM_9', 'TM_12', 'TM_15', 'TM_18', 'PR_9', 'PR_15', 'PR_21',
       'PP_6', 'PP_9', 'PP_12', 'PP_15', 'PP_18', 'HM_6', 'HM_9', 'HM_12',
       'HM_15', 'HM_18', 'WS_6', 'WS_9', 'WS_12', 'WS_15', 'WS_18', 'SK_6',
       'SK_9', 'SK_12', 'SK_5', 'SK_18'],
      dtype='object')

In [230]:
# select the intersection dates between load and weather_forecast_original_df


pv = pd.read_csv('./processed_data/pv/week_2021_PV.csv')
load = pd.read_csv("./processed_data/week_2021_load.csv", index_col = 0)

load_index = load.index
pv.rename(columns = {'Unnamed: 0': 'date'}, inplace = True)
pv = pv.set_index('date')
pv = pv.reindex(load_index)

# the df doesn't have the exact date as their value, so make this
# flag 1 means the date is
weather_forecast_original_df['flag'] = 0

# month list of months having only 30 days
thirty_month = [4, 6, 9, 11]

cnt = 0
for month in range(1, 13):
    if month == 2:
        end_date = 28
    elif month in thirty_month:
        end_date = 30
    else:
        end_date = 31
        
    for day in range(1, end_date+1):
        date = '21{0:0>2}{1:0>2}'.format(month, day)
        if int(date) in load_index:
            weather_forecast_original_df['flag'].iloc[cnt] = 1
        cnt+=1

C:\Users\user\AppData\Local\Temp\ipykernel_6916\2493998530.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_forecast_original_df['flag'].iloc[cnt] = 1


In [231]:
weather_forecast_original_df = weather_forecast_original_df[weather_forecast_original_df['flag'] != 0]
weather_forecast_original_df = weather_forecast_original_df.drop(['flag'], axis=1).set_index(load_index)
weather_forecast_original_df.to_csv('./processed_data/pv_forecast_forecast_model_features.csv', index=True)

In [233]:
weather_forecast_original_df

,TM_6,TM_9,TM_12,TM_15,TM_18,PR_9,PR_15,PR_21,PP_6,PP_9,...,WS_6,WS_9,WS_12,WS_15,WS_18,SK_6,SK_9,SK_12,SK_5,SK_18
210104,-1.0,-2.0,-4.0,-5.0,-6.0,0.0,0.0,0.0,30.0,30.0,...,6.1,3.3,2.7,2.1,2.8,4.0,4.0,4.0,4.0,4.0
210105,0.0,-2.0,-2.0,-2.0,-5.0,0.0,1.0,5.0,30.0,30.0,...,2.4,1.1,2.5,6.0,6.6,4.0,4.0,4.0,4.0,4.0
210106,-7.0,-9.0,-11.0,-11.0,-14.0,5.0,10.0,5.0,90.0,90.0,...,6.5,4.3,2.3,2.1,2.9,4.0,4.0,4.0,4.0,4.0
210107,-7.0,-10.0,-12.0,-13.0,-13.0,0.0,1.0,1.0,30.0,10.0,...,3.7,1.3,1.3,1.6,2.2,4.0,1.0,1.0,3.0,4.0
210108,-3.0,-5.0,-10.0,-12.0,-10.0,0.0,0.0,0.0,20.0,30.0,...,4.7,4.9,1.9,1.3,1.5,3.0,4.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211221,11.0,5.0,0.0,-2.0,-3.0,0.0,0.0,0.0,0.0,0.0,...,2.7,1.5,1.4,1.4,1.9,1.0,1.0,1.0,1.0,1.0
211227,8.0,4.0,1.0,0.0,-1.0,0.0,0.0,0.0,30.0,30.0,...,2.6,1.3,1.1,1.4,1.2,4.0,4.0,4.0,4.0,4.0
211228,8.0,5.0,3.0,1.0,0.0,0.0,0.0,0.0,30.0,30.0,...,4.7,2.0,3.2,2.8,3.0,4.0,4.0,4.0,4.0,4.0
211229,4.0,2.0,0.0,-2.0,-4.0,0.0,0.0,0.0,60.0,30.0,...,6.5,3.1,3.3,3.7,4.2,4.0,4.0,4.0,4.0,4.0


In [234]:
pv = pd.read_csv('./processed_data/pv/week_2021_PV.csv')
pv.rename(columns = {'Unnamed: 0': 'date'}, inplace = True)
pv = pv.set_index('date')
pv = pv.reindex(load_index)

In [223]:
# pv 발전량이 6시부터 발생하는 것을 확인
# 211224 데이터가 비정상적이므로 사용할 데이터에서 1224일의 데이터는 기각
pv.loc[pv['5'] != 0.0]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23


In [283]:
# pv 발전량은 21시 이후에는 전혀 없는 것을 확인
pv.loc[pv['21'] != 0.0]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23


In [280]:
# pv 24h + 28 weather features(5 temp, 3 precip, 5 prob_precip, 5 humid, 5 wind speed, 5 sky)
comb_df = pv.join(weather_forecast_original_df)

In [281]:
comb_df.to_csv('./processed_data/pv_forecastweather_feature.csv', index = True)